In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, median_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from scipy.stats import *
import h3
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from datetime import datetime

In [2]:
columns = ['accepted_driver_id', 'created_at', 'passenger_id', 'source_lat',
       'source_lng', 'destination_lat', 'destination_lng', 'eta', 'provider',
       'ata', 'id', 'city']
rides = pd.read_csv("../rides_2022-09-07_17to19.csv", names=columns)
rides.rename(columns={'id': 'ride_id'}, inplace=True)
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city
count,2.164000e+04,2.164000e+04,21640.000000,21640.000000,21640.000000,21640.000000,21640.000000,21640.000000,2.164000e+04,21640.0
mean,2.014121e+06,2.350918e+07,35.694906,51.401833,35.695767,51.398042,959.328420,1180.374122,4.916790e+09,1.0
std,1.438191e+06,1.800039e+07,0.096689,0.142701,0.096236,0.150727,697.703745,1001.170375,3.622046e+05,0.0
min,2.140000e+02,6.370000e+02,35.209118,50.902416,34.623146,50.670673,1.000000,120.000000,4.916144e+09,1.0
25%,5.546260e+05,4.778300e+06,35.664794,51.331205,35.663613,51.330386,455.000000,488.750000,4.916470e+09,1.0
50%,1.998809e+06,2.356817e+07,35.724831,51.405758,35.725157,51.404190,768.000000,871.000000,4.916806e+09,1.0
75%,3.341234e+06,3.810790e+07,35.756205,51.468071,35.755826,51.471036,1256.000000,1521.250000,4.917168e+09,1.0
max,4.321580e+06,5.728358e+07,35.832058,52.787960,35.966866,52.776981,7160.000000,10212.000000,4.917451e+09,1.0


In [3]:
columns = ['ride_id','driver_id','a_t_a_result.arrival_a_t_a','a_t_a_result.boarding_a_t_a','a_t_a_result.ride_a_t_a','a_t_a_result.arrival_probe_result.probe.point.lat','a_t_a_result.arrival_probe_result.probe.point.lon','a_t_a_result.arrival_probe_result.probe.timestamp','a_t_a_result.arrival_probe_result.confidence','a_t_a_result.arrival_probe_result.h3_index', 'a_t_a_result.arrival_probe_result.k_ring_level','a_t_a_result.boarding_probe_result.probe.point.lat','a_t_a_result.boarding_probe_result.probe.point.lon','a_t_a_result.boarding_probe_result.probe.timestamp','a_t_a_result.boarding_probe_result.confidence','a_t_a_result.boarding_probe_result.h3_index','a_t_a_result.boarding_probe_result.k_ring_level','a_t_a_result.final_destination_probe_result.probe.point.lat','a_t_a_result.final_destination_probe_result.probe.point.lon','a_t_a_result.final_destination_probe_result.probe.timestamp','a_t_a_result.final_destination_probe_result.confidence','a_t_a_result.final_destination_probe_result.h3_index','a_t_a_result.final_destination_probe_result.k_ring_level','a_t_a_result.destination_probe_result.probe.point.lat','a_t_a_result.destination_probe_result.probe.point.lon','a_t_a_result.destination_probe_result.probe.timestamp','a_t_a_result.destination_probe_result.confidence','a_t_a_result.destination_probe_result.h3_index','a_t_a_result.destination_probe_result.k_ring_level','a_t_a_result.extra_destination_probe_result.probe.point.lat','a_t_a_result.extra_destination_probe_result.probe.point.lon','a_t_a_result.extra_destination_probe_result.probe.timestamp','a_t_a_result.extra_destination_probe_result.confidence','a_t_a_result.extra_destination_probe_result.h3_index','a_t_a_result.extra_destination_probe_result.k_ring_level','pickup_a_d_d_result.distance','pickup_a_d_d_result.confidence','pickup_a_d_d_result.route_ratio','pickup_a_d_d_result.g_p_s_ratio','ride_a_d_d_result.distance','ride_a_d_d_result.confidence','ride_a_d_d_result.route_ratio','ride_a_d_d_result.g_p_s_ratio','total_a_d_d_confidence','in_ride_allotment','e_d_d','clickhouse_time','hash']
khatkesh = pd.read_csv("../khatkesh_2022-09-07_17to20.csv", names=columns)
khatkesh.describe()

,ride_id,driver_id,a_t_a_result.arrival_a_t_a,a_t_a_result.boarding_a_t_a,a_t_a_result.ride_a_t_a,a_t_a_result.arrival_probe_result.probe.point.lat,a_t_a_result.arrival_probe_result.probe.point.lon,a_t_a_result.arrival_probe_result.probe.timestamp,a_t_a_result.arrival_probe_result.confidence,a_t_a_result.arrival_probe_result.k_ring_level,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,9.043440e+05,9.043440e+05,904344.000000,9.043440e+05,9.043440e+05,904344.000000,904344.000000,9.043440e+05,904344.000000,904344.000000,...,904344.000000,904344.000000,904344.000000,904344.000000,904344.000000,904344.000000,904344.000000,904344.000000,904344.000000,9.043440e+05
mean,4.916829e+09,2.386208e+06,190.650374,1.321786e+16,4.776094e+07,34.570586,51.898873,2.223376e+15,0.730997,1.171796,...,0.000355,0.014423,13.722454,0.857881,0.000103,0.705383,0.867802,0.041309,8.421349,9.222961e+18
std,6.618209e+05,1.334830e+06,154.364975,4.936112e+17,1.744911e+09,2.563067,3.529245,2.025071e+17,0.342530,1.443084,...,0.016809,0.016305,31.896299,0.190258,0.004796,0.352332,0.180274,0.303695,11.726985,5.326117e+18
min,4.719714e+09,4.300000e+01,0.000000,0.000000e+00,1.000000e+00,25.277533,44.830383,1.662278e+09,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.227536e+12
25%,4.916407e+09,1.225052e+06,90.000000,8.000000e+00,4.260000e+02,32.678195,50.831985,1.662556e+09,0.591716,0.000000,...,0.000000,0.005263,4.634000,0.742794,0.000000,0.452830,0.785107,0.000000,2.802000,4.611770e+18
50%,4.916829e+09,2.450271e+06,172.000000,3.600000e+01,7.320000e+02,35.705357,51.408218,1.662560e+09,0.769231,1.000000,...,0.000000,0.011111,8.734000,0.963577,0.000000,0.895294,0.961827,0.000000,5.228000,9.221640e+18
75%,4.917255e+09,3.619625e+06,266.000000,1.250000e+02,1.260000e+03,36.254910,52.519666,1.662563e+09,1.000000,2.000000,...,0.000000,0.019608,16.544001,1.000000,0.000000,0.985075,1.000000,0.000000,9.960000,1.383335e+19
max,4.917849e+09,4.321993e+06,19555.000000,1.844674e+19,6.379816e+10,39.652897,61.169842,1.844674e+19,1.000000,4.000000,...,1.000000,1.000000,25057.876953,1.000000,1.000000,1.000000,1.000000,40.573002,713.307007,1.844674e+19


In [4]:
rides_khatkesh = pd.merge(rides, khatkesh, on="ride_id")
rides_khatkesh.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,2.164000e+04,2.164000e+04,21640.000000,21640.000000,21640.000000,21640.000000,21640.000000,21640.000000,2.164000e+04,21640.0,...,21640.000000,21640.000000,21640.000000,21640.000000,21640.000000,21640.000000,21640.000000,21640.000000,21640.000000,2.164000e+04
mean,2.014121e+06,2.350918e+07,35.694906,51.401833,35.695767,51.398042,959.328420,1180.374122,4.916790e+09,1.0,...,0.000097,0.013284,14.196197,0.874834,0.000072,0.731725,0.882592,0.013974,8.582726,9.235779e+18
std,1.438191e+06,1.800039e+07,0.096689,0.142701,0.096236,0.150727,697.703745,1001.170375,3.622046e+05,0.0,...,0.007278,0.013235,14.563214,0.174516,0.002420,0.332054,0.165147,0.145397,9.063319,5.317829e+18
min,2.140000e+02,6.370000e+02,35.209118,50.902416,34.623146,50.670673,1.000000,120.000000,4.916144e+09,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.230000,1.744850e+14
25%,5.546260e+05,4.778300e+06,35.664794,51.331205,35.663613,51.330386,455.000000,488.750000,4.916470e+09,1.0,...,0.000000,0.004926,4.817000,0.799384,0.000000,0.540299,0.822309,0.000000,2.886000,4.707043e+18
50%,1.998809e+06,2.356817e+07,35.724831,51.405758,35.725157,51.404190,768.000000,871.000000,4.916806e+09,1.0,...,0.000000,0.009709,9.597000,0.968930,0.000000,0.909091,0.966240,0.000000,5.764000,9.225572e+18
75%,3.341234e+06,3.810790e+07,35.756205,51.468071,35.755826,51.471036,1256.000000,1521.250000,4.917168e+09,1.0,...,0.000000,0.017857,18.343750,1.000000,0.000000,0.982143,1.000000,0.000000,10.967250,1.383046e+19
max,4.321580e+06,5.728358e+07,35.832058,52.787960,35.966866,52.776981,7160.000000,10212.000000,4.917451e+09,1.0,...,1.000000,0.500000,208.685303,1.000000,0.218497,1.000000,1.000000,6.106000,146.891998,1.844594e+19


In [5]:
rides = rides_khatkesh[(rides_khatkesh['eta'] > 180) &
                       (rides_khatkesh['eta'] < 10800) &
                       (rides_khatkesh['a_t_a_result.ride_a_t_a'] > 180) &
                       (rides_khatkesh['a_t_a_result.ride_a_t_a'] < 10800)]
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,2.072200e+04,2.072200e+04,20722.000000,20722.000000,20722.000000,20722.000000,20722.000000,20722.000000,2.072200e+04,20722.0,...,20722.000000,20722.000000,20722.000000,20722.000000,20722.000000,20722.000000,20722.000000,20722.000000,20722.000000,2.072200e+04
mean,2.012896e+06,2.332228e+07,35.695997,51.401132,35.696876,51.397198,994.036724,1223.299151,4.916788e+09,1.0,...,0.000102,0.012138,14.744160,0.875961,0.000055,0.733544,0.883888,0.013753,8.912575,9.221711e+18
std,1.437859e+06,1.798630e+07,0.095773,0.139905,0.095282,0.148431,692.245732,1001.188162,3.614905e+05,0.0,...,0.007438,0.009986,14.639422,0.170699,0.001651,0.330188,0.161545,0.145405,9.117036,5.307986e+18
min,2.140000e+02,6.370000e+02,35.209118,50.902416,34.623146,50.670673,181.000000,120.000000,4.916144e+09,1.0,...,0.000000,0.000000,0.178000,0.000000,0.000000,0.000000,0.056233,0.000000,0.612000,1.744850e+14
25%,5.525532e+05,4.648918e+06,35.667142,51.331513,35.665444,51.330924,490.250000,530.000000,4.916469e+09,1.0,...,0.000000,0.004950,5.299121,0.801997,0.000000,0.546479,0.823675,0.000000,3.167000,4.718672e+18
50%,1.998809e+06,2.324834e+07,35.725134,51.405533,35.725451,51.403778,799.500000,914.000000,4.916803e+09,1.0,...,0.000000,0.009524,10.134500,0.967261,0.000000,0.910714,0.965592,0.000000,6.084000,9.205366e+18
75%,3.337930e+06,3.791717e+07,35.756536,51.466314,35.756103,51.469666,1284.000000,1565.000000,4.917162e+09,1.0,...,0.000000,0.016949,18.877533,1.000000,0.000000,0.981308,1.000000,0.000000,11.271250,1.379651e+19
max,4.321580e+06,5.728355e+07,35.829247,52.787960,35.966866,52.776981,7160.000000,10212.000000,4.917427e+09,1.0,...,1.000000,0.052632,208.685303,1.000000,0.136772,1.000000,1.000000,6.106000,146.891998,1.844594e+19


In [6]:
rides = rides[rides['ride_a_d_d_result.confidence'] > 0.6]
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,1.792100e+04,1.792100e+04,17921.000000,17921.000000,17921.000000,17921.000000,17921.000000,17921.000000,1.792100e+04,17921.0,...,17921.000000,17921.000000,17921.000000,17921.000000,17921.000000,17921.000000,17921.000000,17921.000000,17921.000000,1.792100e+04
mean,2.027443e+06,2.330538e+07,35.695309,51.400976,35.696058,51.397198,985.030132,1211.580269,4.916789e+09,1.0,...,0.000089,0.013050,15.422408,0.931638,0.000028,0.826441,0.936309,0.015543,8.784296,9.220453e+18
std,1.440083e+06,1.798861e+07,0.096610,0.140641,0.096003,0.148787,684.459085,990.954555,3.618643e+05,0.0,...,0.007673,0.009947,14.972229,0.102409,0.001019,0.232236,0.092453,0.155156,8.880589,5.306038e+18
min,2.140000e+02,6.370000e+02,35.209118,50.902416,35.212894,50.670673,181.000000,120.000000,4.916144e+09,1.0,...,0.000000,0.000000,0.178000,0.600080,0.000000,0.000000,0.407188,0.000000,0.612000,1.744850e+14
25%,5.658690e+05,4.638305e+06,35.665936,51.331089,35.664612,51.330353,487.000000,527.000000,4.916469e+09,1.0,...,0.000000,0.005747,5.632000,0.904499,0.000000,0.752475,0.902570,0.000000,3.140000,4.706854e+18
50%,2.031946e+06,2.324436e+07,35.725250,51.404812,35.725208,51.402977,789.000000,906.000000,4.916805e+09,1.0,...,0.000000,0.010309,10.758000,0.983970,0.000000,0.938650,0.981019,0.000000,5.995000,9.218761e+18
75%,3.350436e+06,3.792664e+07,35.756454,51.466881,35.756027,51.469852,1275.000000,1544.000000,4.917165e+09,1.0,...,0.000000,0.017857,19.841999,1.000000,0.000000,0.986577,1.000000,0.000000,11.175000,1.378909e+19
max,4.321580e+06,5.728355e+07,35.829247,52.787960,35.966866,52.776981,7160.000000,10212.000000,4.917415e+09,1.0,...,1.000000,0.052632,208.685303,1.000000,0.069734,1.000000,1.000000,6.106000,140.410995,1.844594e+19


In [7]:
rides = rides[['eta', 'ata', 'a_t_a_result.ride_a_t_a', 'created_at', 'source_lat', 'source_lng', 'destination_lat', 'destination_lng', 'e_d_d', 'clickhouse_time']]
rides

,eta,ata,a_t_a_result.ride_a_t_a,created_at,source_lat,source_lng,destination_lat,destination_lng,e_d_d,clickhouse_time
0,216,264,227,2022-09-07 19:10:22,35.688171,51.242744,35.688530,51.254330,1.661,2022-09-07 19:18:17
1,690,606,576,2022-09-07 17:07:23,35.450420,50.983334,35.472977,51.068180,11.502,2022-09-07 17:26:42
2,539,661,611,2022-09-07 19:29:16,35.668324,51.458286,35.682491,51.450005,3.054,2022-09-07 19:44:19
4,1948,1942,2104,2022-09-07 18:28:36,35.788334,51.375679,35.704464,51.386269,12.503,2022-09-07 19:10:05
7,614,454,439,2022-09-07 19:47:23,35.597797,51.440342,35.607651,51.429901,6.171,2022-09-07 19:59:02
...,...,...,...,...,...,...,...,...,...,...
21635,198,850,799,2022-09-07 19:36:23,35.742573,51.823948,35.741184,51.818558,4.185,2022-09-07 19:52:16
21636,479,311,257,2022-09-07 18:31:30,35.743279,51.330570,35.738327,51.326656,2.528,2022-09-07 18:40:45
21637,499,516,459,2022-09-07 17:33:19,35.348122,51.636688,35.326210,51.644871,3.041,2022-09-07 17:44:56
21638,249,337,383,2022-09-07 19:00:18,35.558105,51.161251,35.554798,51.153976,1.297,2022-09-07 19:12:30


In [8]:
rides['source_h3_4'] = rides.apply(lambda row: h3.geo_to_h3(row.source_lat, row.source_lng, 4), axis=1)
rides['dest_h3_4'] = rides.apply(lambda row: h3.geo_to_h3(row.destination_lat, row.destination_lng, 4), axis=1)

In [9]:
rides['hour'] = rides.apply(lambda row: datetime.strptime(row.created_at,"%Y-%m-%d %H:%M:%S").hour, axis=1)

In [10]:
# , "e_d_d", , "hour"
eta_edd_h3_hour = rides[["eta", "e_d_d", "source_h3_4", "dest_h3_4"]]
ata = rides["a_t_a_result.ride_a_t_a"]

In [12]:
# eta_edd_h3_hour_train, eta_edd_h3_hour_test, ata_train, ata_test = train_test_split(eta_edd_h3_hour, ata, test_size=0.05, random_state=73)
# print(len(eta_edd_h3_hour_train))
# print(len(eta_edd_h3_hour_test))

17024
897


In [27]:
# dummy_train_rides = pd.get_dummies(eta_edd_h3_hour_train, prefix={'source_h3_4':'source_h3_4',
#                                                        'dest_h3_4':'dest_h3_4'},
#                           drop_first=True)
# dummy_train_rides.describe()

,eta,e_d_d,hour,source_h3_4_842cd59ffffffff,source_h3_4_842cd5bffffffff,source_h3_4_842cf31ffffffff,source_h3_4_842cf33ffffffff,source_h3_4_842cf35ffffffff,source_h3_4_842cf37ffffffff,source_h3_4_842cf3dffffffff,dest_h3_4_842cd59ffffffff,dest_h3_4_842cd5bffffffff,dest_h3_4_842cf07ffffffff,dest_h3_4_842cf31ffffffff,dest_h3_4_842cf33ffffffff,dest_h3_4_842cf35ffffffff,dest_h3_4_842cf37ffffffff,dest_h3_4_842cf39ffffffff,dest_h3_4_842cf3dffffffff
count,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000,17024.000000
mean,985.351974,8.779119,18.000000,0.003466,0.000352,0.847509,0.019972,0.053513,0.026668,0.047580,0.002996,0.000411,0.000117,0.836172,0.020383,0.050576,0.025963,0.002408,0.060150
std,685.621068,8.903845,0.820205,0.058770,0.018771,0.359506,0.139907,0.225060,0.161117,0.212882,0.054653,0.020274,0.010839,0.370130,0.141311,0.219136,0.159031,0.049017,0.237772
min,181.000000,0.612000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,487.000000,3.141000,17.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,790.000000,5.986000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1274.250000,11.164250,19.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7160.000000,140.410995,19.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [28]:
# geo_noise = np.count_nonzero(dummy_train_rides, axis=0) < 10
#
# cols = dummy_train_rides.columns
# for col, noise in zip(cols, geo_noise):
#     if noise:
#         # DOESN'T REINDEX
#         eta_edd_h3_hour_train = eta_edd_h3_hour_train[dummy_train_rides[col] == 0]
#         ata_train = ata_train[dummy_train_rides[col] == 0]
#
# eta_edd_h3_hour_train.describe()

/var/folders/yq/707ysz_j3jlggmtk5clx13rm0000gn/T/ipykernel_13512/1070490425.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  eta_edd_h3_hour_train = eta_edd_h3_hour_train[dummy_train_rides[col] == 0]


,eta,e_d_d,hour
count,17015.000000,17015.000000,17015.000000
mean,984.789480,8.764298,18.000000
std,682.806784,8.810351,0.820136
min,181.000000,0.612000,17.000000
25%,488.000000,3.141000,17.000000
50%,790.000000,5.987000,18.000000
75%,1274.000000,11.164000,19.000000
max,4996.000000,89.037003,19.000000


In [29]:
# print(np.count_nonzero(dummy_train_rides, axis=0))
# print(dummy_train_rides.columns)

[17024 17024 17024    59     6 14428   340   911   454   810    51     7
     2 14235   347   861   442    41  1024]
Index(['eta', 'e_d_d', 'hour', 'source_h3_4_842cd59ffffffff',
       'source_h3_4_842cd5bffffffff', 'source_h3_4_842cf31ffffffff',
       'source_h3_4_842cf33ffffffff', 'source_h3_4_842cf35ffffffff',
       'source_h3_4_842cf37ffffffff', 'source_h3_4_842cf3dffffffff',
       'dest_h3_4_842cd59ffffffff', 'dest_h3_4_842cd5bffffffff',
       'dest_h3_4_842cf07ffffffff', 'dest_h3_4_842cf31ffffffff',
       'dest_h3_4_842cf33ffffffff', 'dest_h3_4_842cf35ffffffff',
       'dest_h3_4_842cf37ffffffff', 'dest_h3_4_842cf39ffffffff',
       'dest_h3_4_842cf3dffffffff'],
      dtype='object')


In [30]:
# ohe = ColumnTransformer(
#     [('OHE', OneHotEncoder(handle_unknown='ignore', sparse=False),['source_h3_4', 'dest_h3_4'])],
#     remainder = 'passthrough'
#     ).fit(eta_edd_h3_hour_train)

In [31]:
# eta_edd_h3_hour_train_transformed = ohe.transform(eta_edd_h3_hour_train)

In [32]:
# eta_edd_h3_hour_test_transformed = ohe.transform(eta_edd_h3_hour_test)

In [33]:
# reg = LinearRegression().fit(eta_edd_h3_hour_train_transformed, ata_train)

In [34]:
# print(reg.coef_)
# print(reg.intercept_)

[ -54.24779738  188.41495268  -33.65027002  124.50875181  -89.03181301
  -86.6401199   -49.35370418   99.00226493 -130.2629327    -1.0106306
 -197.7024029     6.6005062    77.90278741  181.93631447  -36.46590682
    1.41462893   -9.8332009   -99.6436114 ]
1734.7242255538758


In [35]:
# ata_train_pred = reg.predict(eta_edd_h3_hour_train_transformed)
# ata_test_pred = reg.predict(eta_edd_h3_hour_test_transformed)

In [36]:
# eta_edd_h3_hour_train_transformed

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.24000000e+02, 1.96700001e+00, 1.80000000e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        5.89000000e+02, 7.10900021e+00, 1.70000000e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        1.20100000e+03, 7.93599987e+00, 1.80000000e+01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        1.48900000e+03, 1.47220001e+01, 1.90000000e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        3.86000000e+02, 1.68599999e+00, 1.90000000e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        1.30600000e+03, 1.36320000e+01, 1.80000000e+01]])

In [37]:
# print("ETA train MAE", mean_absolute_error(ata_train, eta_edd_h3_hour_train_transformed[:, 15]))
# print("prediction train MAE", mean_absolute_error(ata_train, ata_train_pred))
#
# print("ETA test MAE ", mean_absolute_error(ata_test, eta_edd_h3_hour_test_transformed[:, 15]))
# print("prediction test MAE", mean_absolute_error(ata_test, ata_test_pred))
#
#
# print("ETA train MSE", mean_squared_error(ata_train, eta_edd_h3_hour_train_transformed[:, 15]))
# print("prediction train MSE", mean_squared_error(ata_train, ata_train_pred))
#
# print("ETA test MSE ", mean_squared_error(ata_test, eta_edd_h3_hour_test_transformed[:, 15]))
# print("prediction test MSE", mean_squared_error(ata_test, ata_test_pred))
#
#
# print("ETA train explained_variance_score", explained_variance_score(ata_train, eta_edd_h3_hour_train_transformed[:, 15]))
# print("prediction train explained_variance_score", explained_variance_score(ata_train, ata_train_pred))
#
# print("ETA test explained_variance_score ", explained_variance_score(ata_test, eta_edd_h3_hour_test_transformed[:, 15]))
# print("prediction test explained_variance_score", explained_variance_score(ata_test, ata_test_pred))
#
#
# print("ETA train median_absolute_error", median_absolute_error(ata_train, eta_edd_h3_hour_train_transformed[:, 15]))
# print("prediction train median_absolute_error", median_absolute_error(ata_train, ata_train_pred))
#
# print("ETA test median_absolute_error ", median_absolute_error(ata_test, eta_edd_h3_hour_test_transformed[:, 15]))
# print("prediction test median_absolute_error", median_absolute_error(ata_test, ata_test_pred))
#
#
# print("ETA train r2_score", r2_score(ata_train, eta_edd_h3_hour_train_transformed[:, 15]))
# print("prediction train r2_score", r2_score(ata_train, ata_train_pred))
#
# print("ETA test r2_score ", r2_score(ata_test, eta_edd_h3_hour_test_transformed[:, 15]))
# print("prediction test r2_score", r2_score(ata_test, ata_test_pred))
#
#
# print("ETA train MAPE", mean_absolute_percentage_error(ata_train, eta_edd_h3_hour_train_transformed[:, 15]))
# print("prediction train MAPE", mean_absolute_percentage_error(ata_train, ata_train_pred))
#
# print("ETA test MAPE ", mean_absolute_percentage_error(ata_test, eta_edd_h3_hour_test_transformed[:, 15]))
# print("prediction test MAPE", mean_absolute_percentage_error(ata_test, ata_test_pred))

ETA train MAE 282.2007640317367
prediction train MAE 239.33373802397207
ETA test MAE  278.0133779264214
prediction test MAE 244.23609892246478
ETA train MSE 240555.79124302085
prediction train MSE 133363.80261080217
ETA test MSE  290479.19509476033
prediction test MSE 184992.84656882097
ETA train explained_variance_score 0.8013651351429689
prediction train explained_variance_score 0.8609091630246613
ETA test explained_variance_score  0.7525599622257961
prediction test explained_variance_score 0.812445523189655
ETA train median_absolute_error 135.0
prediction train median_absolute_error 160.5293926710724
ETA test median_absolute_error  131.0
prediction test median_absolute_error 163.47580574383073
ETA train r2_score 0.7491140347812297
prediction train r2_score 0.8609091630246613
ETA test r2_score  0.7054884345188464
prediction test r2_score 0.8124391220926338
ETA train MAPE 0.2080388500806881
prediction train MAPE 0.24096110751814095
ETA test MAPE  0.20487941597678036
prediction test MA

In [11]:
rides_p4 = rides[rides['eta'] > 1800]
rides_p4

,eta,ata,a_t_a_result.ride_a_t_a,created_at,source_lat,source_lng,destination_lat,destination_lng,e_d_d,clickhouse_time,source_h3_4,dest_h3_4,hour
4,1948,1942,2104,2022-09-07 18:28:36,35.788334,51.375679,35.704464,51.386269,12.503000,2022-09-07 19:10:05,842cf31ffffffff,842cf31ffffffff,18
13,2034,2432,2425,2022-09-07 18:52:05,35.769527,51.375660,35.749249,51.256992,17.659000,2022-09-07 19:39:46,842cf31ffffffff,842cf31ffffffff,18
14,2755,3106,3294,2022-09-07 18:20:23,35.646183,51.301712,35.715679,51.431812,22.749001,2022-09-07 19:30:40,842cf31ffffffff,842cf31ffffffff,18
19,2688,2753,2748,2022-09-07 18:31:32,35.728317,51.434978,35.724007,51.318825,12.726000,2022-09-07 19:28:29,842cf31ffffffff,842cf31ffffffff,18
20,2668,2632,2627,2022-09-07 18:24:03,35.747692,51.222412,35.684402,51.015369,28.212000,2022-09-07 19:12:30,842cf31ffffffff,842cf3dffffffff,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21555,3027,3701,3931,2022-09-07 17:16:50,35.638645,51.364239,35.655308,51.055885,39.138000,2022-09-07 18:25:04,842cf31ffffffff,842cf3dffffffff,17
21581,1940,2084,2086,2022-09-07 18:50:23,35.763355,51.304192,35.743538,51.079121,29.125999,2022-09-07 19:29:38,842cf31ffffffff,842cf3dffffffff,18
21588,2881,3874,3716,2022-09-07 17:47:31,35.633251,51.381866,35.807922,51.464828,40.250999,2022-09-07 19:03:24,842cf31ffffffff,842cf31ffffffff,17
21606,2542,3706,3684,2022-09-07 17:32:18,35.719128,51.330307,35.784294,51.455704,21.486000,2022-09-07 18:41:56,842cf31ffffffff,842cf31ffffffff,17


In [12]:
rides_p4.describe()

,eta,ata,a_t_a_result.ride_a_t_a,source_lat,source_lng,destination_lat,destination_lng,e_d_d,hour
count,2191.000000,2191.000000,2191.000000,2191.000000,2191.000000,2191.000000,2191.000000,2191.000000,2191.000000
mean,2425.805112,3148.674578,3163.753994,35.704695,51.390758,35.701461,51.355991,25.583150,17.642173
std,551.905195,1059.248791,1023.100914,0.093927,0.118610,0.088278,0.174147,13.055328,0.616168
min,1801.000000,577.000000,593.000000,35.299927,50.911144,35.305145,50.670673,5.102000,17.000000
25%,2016.000000,2391.000000,2429.000000,35.674210,51.335310,35.662163,51.306723,16.116000,17.000000
50%,2288.000000,2976.000000,2987.000000,35.728794,51.402493,35.719696,51.386127,21.615000,18.000000
75%,2680.500000,3707.000000,3693.500000,35.767307,51.443844,35.756796,51.455122,32.015999,18.000000
max,7160.000000,10212.000000,9930.000000,35.829247,51.909485,35.966866,52.776310,140.410995,19.000000


In [35]:
# , "e_d_d",
p4_eta_edd_h3_hour = rides_p4[["eta", "source_h3_4", "dest_h3_4", "hour"]]
p4_ata = rides_p4["a_t_a_result.ride_a_t_a"]

In [36]:
p4_eta_edd_h3_hour_train, p4_eta_edd_h3_hour_test, p4_ata_train, p4_ata_test = train_test_split(p4_eta_edd_h3_hour, p4_ata, test_size=0.05, random_state=73)
print(len(p4_eta_edd_h3_hour_train))
print(len(p4_eta_edd_h3_hour_test))

2081
110


In [37]:
p4_dummy_train_rides = pd.get_dummies(p4_eta_edd_h3_hour_train, prefix={'source_h3_4': 'source_h3_4',
                                                             'dest_h3_4': 'dest_h3_4'},
                                   drop_first=True)
p4_dummy_train_rides.describe()

,eta,hour,source_h3_4_842cf31ffffffff,source_h3_4_842cf33ffffffff,source_h3_4_842cf35ffffffff,source_h3_4_842cf37ffffffff,source_h3_4_842cf3dffffffff,dest_h3_4_842cd5bffffffff,dest_h3_4_842cf07ffffffff,dest_h3_4_842cf31ffffffff,dest_h3_4_842cf33ffffffff,dest_h3_4_842cf35ffffffff,dest_h3_4_842cf37ffffffff,dest_h3_4_842cf39ffffffff,dest_h3_4_842cf3dffffffff
count,2081.000000,2081.000000,2081.000000,2081.000000,2081.000000,2081.000000,2081.000000,2081.000000,2081.000000,2081.000000,2081.000000,2081.00000,2081.000000,2081.000000,2081.000000
mean,2421.558385,17.639596,0.871696,0.015858,0.049976,0.018741,0.043248,0.000481,0.000961,0.776550,0.015377,0.04421,0.013455,0.019702,0.127823
std,550.668881,0.617726,0.334508,0.124955,0.217948,0.135641,0.203465,0.021921,0.030994,0.416658,0.123077,0.20561,0.115241,0.139008,0.333973
min,1801.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,2013.000000,17.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,2285.000000,18.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,2676.000000,18.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000
max,7160.000000,19.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


In [38]:
p4_geo_noise = np.count_nonzero(p4_dummy_train_rides, axis=0) < 10

p4_cols = p4_dummy_train_rides.columns
for col, noise in zip(p4_cols, p4_geo_noise):
    if noise:
        # DOESN'T REINDEX
        p4_eta_edd_h3_hour_train = p4_eta_edd_h3_hour_train[p4_dummy_train_rides[col] == 0]
        p4_ata_train = p4_ata_train[p4_dummy_train_rides[col] == 0]

p4_eta_edd_h3_hour_train.describe()

/var/folders/yq/707ysz_j3jlggmtk5clx13rm0000gn/T/ipykernel_1251/2658656946.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p4_eta_edd_h3_hour_train = p4_eta_edd_h3_hour_train[p4_dummy_train_rides[col] == 0]


,eta,hour
count,2078.000000,2078.000000
mean,2417.022618,17.640520
std,536.194724,0.617693
min,1801.000000,17.000000
25%,2013.000000,17.000000
50%,2284.500000,18.000000
75%,2674.500000,18.000000
max,4996.000000,19.000000


In [39]:
print(np.count_nonzero(p4_dummy_train_rides, axis=0))
print(p4_dummy_train_rides.columns)

[2081 2081 1814   33  104   39   90    1    2 1616   32   92   28   41
  266]
Index(['eta', 'hour', 'source_h3_4_842cf31ffffffff',
       'source_h3_4_842cf33ffffffff', 'source_h3_4_842cf35ffffffff',
       'source_h3_4_842cf37ffffffff', 'source_h3_4_842cf3dffffffff',
       'dest_h3_4_842cd5bffffffff', 'dest_h3_4_842cf07ffffffff',
       'dest_h3_4_842cf31ffffffff', 'dest_h3_4_842cf33ffffffff',
       'dest_h3_4_842cf35ffffffff', 'dest_h3_4_842cf37ffffffff',
       'dest_h3_4_842cf39ffffffff', 'dest_h3_4_842cf3dffffffff'],
      dtype='object')


In [40]:
p4_ohe = ColumnTransformer(
    [('OHE', OneHotEncoder(handle_unknown='ignore', sparse=False), ['source_h3_4', 'dest_h3_4'])],
    remainder='passthrough'
).fit(p4_eta_edd_h3_hour_train)

In [41]:
p4_eta_edd_h3_hour_train_transformed = p4_ohe.transform(p4_eta_edd_h3_hour_train)
p4_eta_edd_h3_hour_test_transformed = p4_ohe.transform(p4_eta_edd_h3_hour_test)

In [42]:
p4_reg = LinearRegression().fit(p4_eta_edd_h3_hour_train_transformed, p4_ata_train)
print(p4_reg.coef_)
print(p4_reg.intercept_)

[ 193.80608842  307.55735963  -36.15459385 -145.63361408 -258.54478093
  -61.03045919 -373.19895841  381.34337198 -353.98614156   16.14755201
  234.42520003  141.34158402  -46.07260806    1.38587456 -462.20694103]
7416.846760447172


In [43]:
p4_ata_train_pred = p4_reg.predict(p4_eta_edd_h3_hour_train_transformed)
p4_ata_test_pred = p4_reg.predict(p4_eta_edd_h3_hour_test_transformed)

In [44]:
p4_eta_edd_h3_hour_train_transformed

array([[0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 2.251e+03,
        1.900e+01],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 2.359e+03,
        1.700e+01],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 1.802e+03,
        1.800e+01],
       ...,
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 1.883e+03,
        1.700e+01],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 1.925e+03,
        1.700e+01],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 2.053e+03,
        1.800e+01]])

In [45]:
print("ETA train MAE", mean_absolute_error(p4_ata_train, p4_eta_edd_h3_hour_train_transformed[:, 13]))
print("prediction train MAE", mean_absolute_error(p4_ata_train, p4_ata_train_pred))

print("ETA test MAE ", mean_absolute_error(p4_ata_test, p4_eta_edd_h3_hour_test_transformed[:, 13]))
print("prediction test MAE", mean_absolute_error(p4_ata_test, p4_ata_test_pred))

print("ETA train MSE", mean_squared_error(p4_ata_train, p4_eta_edd_h3_hour_train_transformed[:, 13]))
print("prediction train MSE", mean_squared_error(p4_ata_train, p4_ata_train_pred))

print("ETA test MSE ", mean_squared_error(p4_ata_test, p4_eta_edd_h3_hour_test_transformed[:, 13]))
print("prediction test MSE", mean_squared_error(p4_ata_test, p4_ata_test_pred))

print("ETA train explained_variance_score", explained_variance_score(p4_ata_train, p4_eta_edd_h3_hour_train_transformed[:, 13]))
print("prediction train explained_variance_score", explained_variance_score(p4_ata_train, p4_ata_train_pred))

print("ETA test explained_variance_score ", explained_variance_score(p4_ata_test, p4_eta_edd_h3_hour_test_transformed[:, 13]))
print("prediction test explained_variance_score", explained_variance_score(p4_ata_test, p4_ata_test_pred))

print("ETA train median_absolute_error", median_absolute_error(p4_ata_train, p4_eta_edd_h3_hour_train_transformed[:, 13]))
print("prediction train median_absolute_error", median_absolute_error(p4_ata_train, p4_ata_train_pred))

print("ETA test median_absolute_error ", median_absolute_error(p4_ata_test, p4_eta_edd_h3_hour_test_transformed[:, 13]))
print("prediction test median_absolute_error", median_absolute_error(p4_ata_test, p4_ata_test_pred))

print("ETA train r2_score", r2_score(p4_ata_train, p4_eta_edd_h3_hour_train_transformed[:, 13]))
print("prediction train r2_score", r2_score(p4_ata_train, p4_ata_train_pred))

print("ETA test r2_score ", r2_score(p4_ata_test, p4_eta_edd_h3_hour_test_transformed[:, 13]))
print("prediction test r2_score", r2_score(p4_ata_test, p4_ata_test_pred))

print("ETA train MAPE", mean_absolute_percentage_error(p4_ata_train, p4_eta_edd_h3_hour_train_transformed[:, 13]))
print("prediction train MAPE", mean_absolute_percentage_error(p4_ata_train, p4_ata_train_pred))

print("ETA test MAPE ", mean_absolute_percentage_error(p4_ata_test, p4_eta_edd_h3_hour_test_transformed[:, 13]))
print("prediction test MAPE", mean_absolute_percentage_error(p4_ata_test, p4_ata_test_pred))

ETA train MAE 798.3893166506256
prediction train MAE 471.54426573862924
ETA test MAE  742.1545454545454
prediction test MAE 468.09675937043033
ETA train MSE 1111203.771414822
prediction train MSE 411897.3359578221
ETA test MSE  916444.5181818182
prediction test MSE 340487.3503994411
ETA train explained_variance_score 0.45018431996690533
prediction train explained_variance_score 0.5981240278391844
ETA test explained_variance_score  0.5547478416786853
prediction test explained_variance_score 0.6667504536558913
ETA train median_absolute_error 642.5
prediction train median_absolute_error 370.7770537836168
ETA test median_absolute_error  529.0
prediction test median_absolute_error 401.7086228807973
ETA train r2_score -0.08416844908126375
prediction train r2_score 0.5981240278391844
ETA test r2_score  0.09827939359543225
prediction test r2_score 0.6649830360877815
ETA train MAPE 0.22912438141911642
prediction train MAPE 0.1599664856093615
ETA test MAPE  0.2150539993104505
prediction test MAP

In [1]:
!head ../big_query_24_07.csv

3341521,2022-09-07 19:10:22,17723801,35.68817138671875,51.24274444580078,35.68852996826172,51.254329681396484,216,smapp-same-dc,264,4917107579,1
3955313,2022-09-07 17:07:23,11915471,35.45042037963867,50.983333587646484,35.47297668457031,51.068180084228516,690,smapp-same-dc,606,4916198765,1
182694,2022-09-07 19:29:16,1488366,35.6683235168457,51.45828628540039,35.682491302490234,51.45000457763672,539,smapp-same-dc,661,4917246696,1
2895740,2022-09-07 17:53:19,28913733,35.806636810302734,51.39913558959961,35.654701232910156,51.45022201538086,3446,smapp-same-dc,5007,4916535274,1
770043,2022-09-07 18:28:36,129207,35.788333892822266,51.37567901611328,35.704463958740234,51.386268615722656,1948,smapp-same-dc,1942,4916803953,1
843595,2022-09-07 19:01:13,466911,35.76164627075195,51.39791488647461,35.7697639465332,51.39655685424805,169,smapp-same-dc,141,4917041067,1
4141159,2022-09-07 19:27:29,41101665,35.306297302246094,51.728370666503906,35.304649353027344,51.735328674316406,128,smapp-same-dc,29